## **Algoritmo de filtraje no lineal basado en Operador de Koopman aplicado a epidemiología**

### **Capítulo 2: Kernel Extended Dynamic Mode Decomposition**

**Autor: Diego Olguín.**

**Supervisores: Héctor Ramírez y Axel Osses.**

In [65]:
chapter = "chapter3/"
img_path = "img/content/"+chapter

In [66]:
# Librerías generales
import numpy as np
from scipy import stats
from sklearn.gaussian_process.kernels import Matern

# Plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Librerías propias
import os

# Dependencias de KKKF
from KKKF.DynamicalSystems import DynamicalSystem
from KKKF.kEDMD import KoopmanOperator

In [67]:
# Ajustar directorio para las imagénes
root = "/Users/diegoolguin/Koopman_nonlinear_filter"
if not os.getcwd()==root:
    os.chdir(root)

### **kEDMD en el caso de modelos lineales**

In [68]:
# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 2500
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

# Distribuciones
X_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=3*np.eye(3))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(3))
obs_dist = stats.multivariate_normal(mean=np.zeros(1), cov=1e-7*np.eye(1))

# Datos sintéticos
iters = 50
x0 = np.array([0.1, 0.1, 0.1])

# Tiempo
t = np.arange(iters)

In [69]:
# Función auxiliar para cálculo de kEDMD en el caso lineal
def compute_linear_kedmd(alpha, N):
    # Matriz
    A = np.array([
        [0.01, 0.04, 0.0],
        [0.01, 0.02, alpha],
        [0.0, 0.04, 0.02]
    ])

    # Función de dinámica
    f = lambda x: x + A@x

    # Solución real
    x = np.zeros((iters, nx))
    x[0] = x0

    for i in range(1, iters):
        x[i] = f(x[i-1])

    # Sistema dinámico
    dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

    # Operador de Koopman
    Koop = KoopmanOperator(k, dyn)

    # Se calcula kEDMD
    Koop.compute_edmd(N)
    U, B, phi = Koop.U, Koop.B, Koop.phi

    # Sistema con Koopman
    x_koop = np.zeros((iters, nx))
    x_koop[0] = B@phi(x0)
    z = np.zeros((iters, N))
    z[0] = phi(x0)

    for i in range(1, iters):
        z[i] = U@z[i-1]
        x_koop[i] = B@z[i]

    return x, x_koop

In [ ]:
# Alpha
alpha = -0.3

# Predicción
x, x_koop = compute_linear_kedmd(alpha, N)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='x1 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='x2 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='x3 real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='x1 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='x2 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='x3 predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos, matriz de parámetro {alpha}",
    xaxis_title="Tiempo",
    yaxis_title="Estados"
    )

fig.write_image(img_path+"Linear1.pdf")
fig.show()

In [ ]:
# Alpha
alpha = -0.1

# Predicción
x, x_koop = compute_linear_kedmd(alpha, N)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='x1 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='x2 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='x3 real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='x1 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='x2 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='x3 predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos, matriz de parámetro {alpha}",
    xaxis_title="Tiempo",
    yaxis_title="Estados"
    )

fig.write_image(img_path+"Linear2.pdf")
fig.show()

In [ ]:
# Alpha
alpha = 0.05

# Predicción
x, x_koop = compute_linear_kedmd(alpha, N)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='x1 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='x2 real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='x3 real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='x1 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='x2 predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='x3 predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo lineal con {N} puntos, matriz de parámetro {alpha}",
    xaxis_title="Tiempo",
    yaxis_title="Estados"
    )

fig.write_image(img_path+"Linear3.pdf")
fig.show()

In [59]:
# Alpha a utilizar
alpha = -0.3

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop = compute_linear_kedmd(alpha, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Error de kEDMD para modelo lineal con matriz de parámetro {alpha}",
    xaxis_title="Número de puntos",
    yaxis_title="Error"
    )

fig.write_image(img_path+"Linear1Errors.pdf")

fig.show()

In [ ]:
# Alpha a utilizar
alpha = -0.1

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop = compute_linear_kedmd(alpha, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Error de kEDMD para modelo lineal con matriz de parámetro {alpha}",
    xaxis_title="Número de puntos",
    yaxis_title="Error"
    )

fig.write_image(img_path+"Linear2Errors.pdf")

fig.show()

In [61]:
# Alpha a utilizar
alpha = 0.05

# Número de puntos
Ns = np.arange(100, 3100, 100)

# Errores
errors = np.zeros(len(Ns))

# Cálculo de errores
for i in range(len(Ns)):
    x, x_koop = compute_linear_kedmd(alpha, Ns[i])
    errors[i] = np.linalg.norm(x - x_koop)

# Ajustar una curva como raíz cuadrada a los errores
fit = np.polyfit(np.log(Ns), np.log(errors), 1)

# Plot
fig = go.Figure()

fig.add_trace(go.Scatter(x=Ns, y=errors, name='Errores', mode='markers'))
fig.add_trace(go.Scatter(x=Ns, y=np.exp(fit[1])*Ns**fit[0], name=f'Exponente: {fit[0].round(4)}', mode='lines'))

# Colocar titulo
fig.update_layout(
    title=f"Error de kEDMD para modelo lineal con matriz de parámetro {alpha}",
    xaxis_title="Número de puntos",
    yaxis_title="Error"
    )

fig.write_image(img_path+"Linear3Errors.pdf")

fig.show()

### **kEDMD para modelo SIR**

In [ ]:
# Parámetros del modelo SIR
beta, gamma = 1, 0.3

# Dinámica del sistema
def f(x):
    return np.array([x[0]-beta*x[0]*x[1] ,x[1]+beta*x[0]*x[1] - gamma*x[1], x[2]+gamma*x[1]])

# Observaciones del sistema, irrelevantes para este ejemplo
def g(x):
    return x

# Dimensiones y el kernel
N = 500
nx, ny = 3, 3
k = Matern(length_scale=1e3, nu=0.5)

# Distribuciones
X_dist = stats.dirichlet(alpha=1/3*np.ones(nx))
dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1e-7*np.eye(3))
obs_dist = stats.multivariate_normal(mean=np.zeros(1), cov=1e-7*np.eye(1))

# Sistema dinámico
dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

# Datos sintéticos
iters = 20
x0 = np.array([0.9, 0.1, 0.0])
x = np.zeros((iters, nx))

x[0] = x0

for i in range(1, iters):
    x[i] = f(x[i-1])

# Operador de Koopman
Koop = KoopmanOperator(k, dyn)
Koop.compute_edmd(N)

U, B, phi = Koop.U, Koop.B, Koop.phi

# Sistema con Koopman
x_koop = np.zeros((iters, nx))
x_koop[0] = B@phi(x0)
z = np.zeros((iters, N))
z[0] = phi(x0)

for i in range(1, iters):
    z[i] = U@z[i-1]
    x_koop[i] = B@z[i]

t = np.arange(iters)

# Gráfico
fig = go.Figure()

fig.add_trace(go.Scatter(x=t, y=x[:,0], name='S real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,1], name='I real', mode='markers'))
fig.add_trace(go.Scatter(x=t, y=x[:,2], name='R real', mode='markers'))

fig.add_trace(go.Scatter(x=t, y=x_koop[:,0], name='S predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,1], name='I predicho'))
fig.add_trace(go.Scatter(x=t, y=x_koop[:,2], name='R predicho'))

# Colocar titulo
fig.update_layout(
    title=f"kEDMD para modelo SIR de parámetros ({beta}, {gamma}) con N = {N} puntos",
    xaxis_title="Tiempo",
    yaxis_title="Población normalizada"
    )

fig.write_image(img_path+"SIR.pdf")
fig.show()